# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [ ]:
df = pd.read_json('News_Category_Dataset_v2.json',lines=True)
df.head()

## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [ ]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [ ]:
data[:5]

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [ ]:
model = Word2Vec(data, vector_size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [ ]:
model.train(data, total_examples=model.corpus_count, epochs =10)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [ ]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [ ]:
wv.most_similar('Texas')

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [ ]:
wv.most_similar(negative='Texas')

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [ ]:
wv['Texas']

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [ ]:
wv.vectors

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [ ]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

In [2]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [3]:
df['combined_text'] = df['headline'] + ' ' +  df['short_description']
data = df['combined_text'].map(word_tokenize)

In [6]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

In [7]:
model = Word2Vec(data, vector_size=100, window=5, min_count=1, workers=4)

In [8]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55587921, 67375740)

In [9]:
wv = model.wv

In [10]:
wv.most_similar('Texas')

[('Ohio', 0.8114644885063171),
 ('Oklahoma', 0.8078965544700623),
 ('Oregon', 0.8014804124832153),
 ('Pennsylvania', 0.7994517087936401),
 ('Arkansas', 0.7950179576873779),
 ('Maryland', 0.7922462224960327),
 ('California', 0.7889901995658875),
 ('Georgia', 0.7862942814826965),
 ('Illinois', 0.7814738154411316),
 ('Louisiana', 0.7804436683654785)]

In [11]:
wv.most_similar(negative='Texas')

[('sparks', 0.4906822443008423),
 ('DontLookAway', 0.4733099043369293),
 ('WhyIResist', 0.4418509304523468),
 ('UpdatedXmasCarols', 0.39731213450431824),
 ('heave-ho', 0.39669764041900635),
 ('boastfully', 0.3952792286872864),
 ('Fresh-faced', 0.39376428723335266),
 ('BlackMenSmiling', 0.38762593269348145),
 ('communicates', 0.38180306553840637),
 ('Perdu', 0.37799832224845886)]

In [12]:
wv['Texas']

array([-1.3914683 ,  3.311886  , -0.15944017,  0.09111822, -1.5032307 ,
       -0.38754493,  0.15480153, -0.37126136, -1.6048142 , -0.7741635 ,
       -0.2766719 , -0.29948962,  0.05718844, -0.74343204, -1.4055982 ,
       -1.0026846 ,  2.121728  , -0.10006417, -0.6355478 ,  0.76882803,
        0.65211976,  1.3340045 ,  0.7068402 , -1.1546625 ,  1.0515442 ,
       -2.3399997 , -3.175351  , -0.77824837, -0.7476319 ,  1.0577165 ,
       -0.34116015, -2.1234004 ,  0.07437013, -3.4547565 , -0.2774369 ,
        1.7167621 , -0.08480561, -2.0519297 , -2.098326  , -1.0183151 ,
       -1.6244557 ,  1.161     , -0.8914733 ,  0.33245036,  0.0105246 ,
       -2.68752   , -1.2580638 ,  0.22519466, -0.7443025 , -0.8409663 ,
        0.61483806, -2.2583585 , -1.2382917 , -0.03620287, -0.96071243,
        1.7645047 , -0.5191172 , -0.5383705 , -0.2454814 , -0.5540748 ,
       -0.01751945, -1.1495247 , -2.376259  , -1.1197578 ,  1.7815702 ,
        0.22876157,  2.636911  ,  0.8656151 , -1.0706754 , -0.66

In [13]:
wv.vectors

array([[-9.0182561e-01,  1.5445532e+00, -5.0516415e-01, ...,
         1.5046548e+00, -6.8891126e-01, -8.7242872e-01],
       [ 2.4880871e-01,  7.0364851e-01, -7.2570145e-01, ...,
         2.5200942e+00, -1.4666159e-01,  4.7220039e-01],
       [ 2.6875845e-01,  1.4506550e+00, -9.6761829e-01, ...,
         2.5487826e+00,  9.7916108e-01, -2.6720123e+00],
       ...,
       [-4.5872845e-02, -5.7822019e-02,  5.4373100e-02, ...,
        -4.5515325e-02, -4.5165092e-02,  1.6241204e-02],
       [-1.1567055e-02,  3.2537241e-02, -9.3566082e-02, ...,
        -5.7445034e-02,  3.0604588e-02,  4.5204286e-02],
       [-7.0935652e-02, -1.1935053e-02, -2.5757886e-02, ...,
        -7.9830818e-02, -3.9820820e-02,  3.0214339e-04]], dtype=float32)

In [14]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('ssy', 0.5620831847190857),
 ('ke', 0.5539129972457886),
 ('tch', 0.5500184297561646),
 ('ck', 0.5400114059448242),
 ('cking', 0.5379074215888977),
 ('glug', 0.5353102684020996),
 ('NSYNC', 0.5304405689239502),
 ('cked', 0.5271782875061035),
 ('rn', 0.5267289280891418),
 ('streetstyle', 0.5213373899459839)]